# Introduction

In this notebook, I want to take a deep dive into sampling from a log likelihood with vanilla MCMC. This will help me build my intuition around Bayesian statistics.

## Recap Bayes' Rule

Bayes' rule is as follows:

$$P(H|D)= \frac{P(D|H)P(H)}{P(D)}$$

Here:

- $P(H)$ is the prior distribution on $H$ having not seen the data.
- $P(H|D)$ is the posterior belief on $H$ having seen the data $D$.
- $P(D|H)$ is the likelihood of the data.
- $P(D)$ is the normalizing constant, or the probability of the data.

## Translating $P(X)$s into code

I find I don't _truly_ understand something until I am able to translate it into code. Here, I will attempt to do so.

I will start with the components that I know. 

Firstly, I know how to calculate the likelihood of data given a hypothesis. This involves writing a function that takes in data points, and calculates the sum of log likelihoods (or product of likelihoods) assuming that the samples of data are i.i.d.

In [ ]:
import numpy as np
from scipy.stats import norm

In [ ]:
def norm_loglike(x, **norm_params):
    dist = norm(**norm_params)
    
    return np.sum(dist.logpdf(x))

Let's now use some fake data to test our understanding.

I will generate fake data from a $N(3,1)$ distribution, but calculate the log likelihood under a $N(0,1)$ distribution.

In [ ]:
xs = norm(loc=3, scale=1).rvs(100)

norm_loglike(xs, loc=0, scale=1)

Now, compare that with the log likelihood from the actual distribution.

In [ ]:
norm_loglike(xs, loc=3, scale=1)

Now, we still haven't had any "priors" injected here. Let's try it out with priors placed on the location of the Normal distribution, $m$.

Assume $m \sim N(0,10)$. In probability notation, 

$$P(m) = \frac{1}{\sigma_m \sqrt{2 \pi}} e^\frac{-(m-\mu_m)}{2 \sigma_m^2}$$

In [ ]:
def loglike_prior(m):
    dist = norm(loc=0, scale=10)
    return np.sum(dist.logpdf(m))

Let's test a few prior values, and calculate their log likelihood.

In [ ]:
print(loglike_prior(0))
print(loglike_prior(-1))
print(loglike_prior(1))
print(loglike_prior(-100))
print(loglike_prior(100))

Basically what we would expect. $0$ has the highest log likelihood, and log-likelihood goes down symmetrically.

Now, however, we need to use this prior in conjunction with the likelihood.

In [ ]:
def joint_log_prob(m, xs):
    return loglike_prior(m) + norm_loglike(xs, loc=m, scale=1)

When we multiply the prior probability distribution by the likelihood probability distribution, we are doing nothing more than summing up their log likelihoods.

In [ ]:
lls = []
for m in np.linspace(-10, 10, 100):
    ll = joint_log_prob(m, xs)
    lls.append(ll)

In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.plot(np.linspace(-10, 10, 100), lls)

Let's find the max.

In [ ]:
np.linspace(-10, 10, 100)[lls.index(max(lls))]

Woohoo, this is close to the true value!

## Sampling Version

Let's now do it by _sampling_.

We are going to use the [Metropolis-Hastings algorithm](https://en.wikipedia.org/wiki/Metropolis%E2%80%93Hastings_algorithm).

Briefly, it works as follows:

- Initialize an arbitrary point.
- Choose density to propose new point (we will use $N(m_{t-1}, 1)$).
- For each iteration:
    - Generate candidate new candidate $m_t$.
    - Calculate acceptance ratio. We take advantage of the joint log probability L, by passing in the proposed value of $m_t$ into the logprob function, i.e. $L(m_t)$ and comparing it to $L(m_{t-1})$.
    - Now, we compute the ratio $r = \frac{L(m_t)}{L(m_{t-1})}$. 
    - Generate a new random number on the interval $p \sim U(0, 1)$.
    - Compare $p$ to $r$. 
        - If $p \leq r$, accept $m_t$.
        - If $p \gt r$, reject $m_t$ and continue sampling again with $m_{t-1}$.
        
Let's write it in code.

In [ ]:
# Metropolis-Hastings Sampling
m_prev = np.random.normal(0, 1)


history = dict()
ratio_history = dict()
for i in range(1000):
    history[i] = m_prev
    m_t = np.random.normal(m_prev, 5)
    
    # Compute negative joint log likelihood
    L_t = np.exp(joint_log_prob(m_t, xs))
    L_prev = np.exp(joint_log_prob(m_prev, xs))
    
    ratio = L_t / L_prev
    ratio_history[i] = L_t
    p = np.random.uniform(0, 1)
    if p <= ratio:
        m_prev = m_t
    

In [ ]:
np.array([i for i in history.values()]).mean()

It worked!

Now lies a challenge: How do we design an API that can handle not just `m` as the only variable, but arbitrary numbers of variables that have to be learned?

Let's try this out with a model that has two random variables instead of one. Here, we'll add variance of the likelihood to the list of RVs.

Here:

$$\mu \sim N(0, 10)$$
$$\sigma \sim Exp(2)$$
$$Y \sim N(\mu, \sigma)$$

In [ ]:
from scipy.stats import expon

def normal_prior_loglike(mu):
    return np.sum(norm(0, 10).logpdf(mu))

def sigma_prior_loglike(sigma):
    return np.sum(expon(2).logpdf(sigma))

def data_loglike(mu, sigma, xs):
    return np.sum(norm(mu, sigma).logpdf(xs))

In [ ]:
def model_loglike(mu, sigma, xs):
    return normal_prior_loglike(mu) + sigma_prior_loglike(sigma) + data_loglike(mu, sigma, xs)

In [ ]:
# Metropolis-Hastings Sampling
mu_prev = np.random.normal(0, 1)
sigma_prev = np.random.normal(0, 1)

mu_history = dict()
sigma_history = dict()
ratio_history = dict()

# Let's test this with 100 steps.
for i in range(100):
    mu_history[i] = mu_prev
    sigma_history[i] = sigma_prev
    mu_t = np.random.normal(mu_prev, 5)
    sigma_t = np.random.normal(sigma_prev, 5)
    
    # Compute negative joint log likelihood
    LL_t = model_loglike(mu_t, sigma_t, xs)
    LL_prev = model_loglike(mu_prev, sigma_prev, xs)
    
    ratio = np.exp(LL_t - LL_prev)
    ratio_history[i] = ratio
    p = np.random.uniform(0, 1)
    if p <= ratio:
        m_prev = m_t

In [ ]:
ratio_history

Oh no, we get NaNs and infs in there! Why? Because we passed in invalid values into the logpdf of sigma. 

Sigma is bounded as a positive distribution, so we need to to somehow either:

- Restrict sigma's proposal distribution to be positive, or
- Transform sigma such that it lives on the real line, propose a new value on the real line, and then transform it back to original sigma.

We need to decompose this into its constituent components.

1. We have a `sampler` to sample from the joint log-likelihood, which should accept proposed random variable values and data, and return a scalar.
2. We have a `model` specification, which should accept data and return a log-likelihood function and the random variables values to evaluate.

An API sketch might look like the following: